# Advanced nurse scheduling in CPMpy
In this example, a schedule for a nurse rostering problem is computed. \
Each shift in the roster has a capacity and nurses can have certain preferences.

In [1]:
from cpmpy import *
import random

random.seed(0)


def simple_nurse_rostering(num_nurses, shifts_per_day, capacity_per_shifts, num_days):

    # Define the variables
    max_cap = max(capacity_per_shifts)
    roster_matrix = intvar(0, num_nurses, shape=(shifts_per_day, num_days, max_cap), name="roster")

    # Define the constraints
    model = Model()

    # Constraint: Each shift in a day must be assigned to a different nurse
    for day in range(num_days):
        model += AllDifferentExcept0(roster_matrix[:,day,:])

    # Constraint: Each shift must be fully populated
    for shift, capacity in enumerate(capacity_per_shifts):
        for day in range(num_days):
            model += sum(roster_matrix[shift, day,:] != 0) == capacity
            # TODO: add lexless for each shift
                 
    # Constraint: The last shift of a day cannot have the same nurse as the first shift of the next day
    for day in range(num_days - 1):
        model += (roster_matrix[shifts_per_day - 1, day] != roster_matrix[0, day + 1])
    
    # Make sure fair allocation of shifts
    min_nb_shifts = min([sum(roster_matrix == n) for n in range(num_nurses)])
    max_nb_shifts = max([sum(roster_matrix == n) for n in range(num_nurses)])
    model.minimize(max_nb_shifts - min_nb_shifts)
    
    return model, (roster_matrix,)


num_nurses = 10
shifts_per_day = 3
capacity_per_shift = [3,4,2]
num_days = 7

model, (roster_matrix,) = simple_nurse_rostering(num_nurses, shifts_per_day, capacity_per_shift, num_days)

assert model.solve()

In [2]:
def generate_preferences(nb_preferences, n_nurses, roster_matrix):
    
    preferences = []
    
    n_shifts, n_days, _ = roster_matrix.shape
    
    for _ in range(nb_preferences):
        # pick one of the preference types
        p = random.random()
        if p <= 0.5:
            # Nurse does not want to work with another nurse
            nurse1,nurse2 = random.sample(list(range(1,n_nurses+1)), k=2)
            # TODO: add global cardinality count
        else:
            # Nurse does not want to work on a specific day
            nurse = random.randint(0, n_nurses)
            day = random.randint(0,n_days)
            yield all(shift != nurse for shift in roster_matrix[:,day-1,:].flatten())            
            

for pref in generate_preferences(5, num_nurses, roster_matrix):
    model += pref
    
assert model.solve()


In [13]:
import pandas as pd

def weekday(i):
    if i % 7 == 0: return "Monday"
    if i % 7 == 1: return "Tuesday"
    if i % 7 == 2: return "Wednesday"
    if i % 7 == 3: return "Thursday"
    if i % 7 == 4: return "Friday"
    if i % 7 == 5: return "Saturday"
    if i % 7 == 6: return "Sunday"
    

def make_pretty_roster(styler):

    styler.set_caption("Nurse roster")
    styler.set_table_styles([{'selector': 'th.col_heading', 'props': 'text-align: center;'}])
    
    styler.format_index(lambda v: f"Shift {v+1}")
    styler.format_index(weekday, axis="columns", level=0)   
    styler.background_gradient(axis=None, vmin=0, vmax=num_nurses, cmap="Set3") 

    styler.format(lambda v : '' if v == 0 else v)
    styler.set_properties(subset=[(day,0) for day in range(num_days)], **{'border-left': '2px solid #000066'})

    styler.set_properties(**{'text-align': 'center'})
    display(styler.hide(axis='columns', level=1)
)

In [14]:

reshaped_roster = roster_matrix.value().reshape((shifts_per_day, max(capacity_per_shift)*num_days))

cols = pd.MultiIndex.from_product([range(num_days), range(max(capacity_per_shift))])
pd_roster = pd.DataFrame(reshaped_roster, columns=cols)

pd_roster.style.pipe(make_pretty_roster)
